### EEVE Model Basic Testing

Code to test the baseline functionality of the EEVE model by yanolja.

In [1]:
# Import modules 

from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
import torch

/home/kimjaehyun/anaconda3/envs/nlp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load model

model = AutoModelForCausalLM.from_pretrained(
    "yanolja/EEVE-Korean-Instruct-10.8B-v1.0",
    device_map="auto",
    torch_dtype=torch.float16, # half-precision to speed up inference
)
tokenizer = AutoTokenizer.from_pretrained("yanolja/EEVE-Korean-Instruct-10.8B-v1.0",
                                          device_map="auto")

Loading checkpoint shards: 100%|██████████| 5/5 [00:46<00:00,  9.29s/it]


In [3]:
# Set text

prompt_template = "A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions.\nHuman: {prompt}\nAssistant:\n"
text = "한국의 수도는 어디인가요? 아래 선택지 중 골라주세요.\n\n(A) 경성\n(B) 부산\n(C) 평양\n(D) 서울\n(E) 전주"

In [4]:
# Process through model and obtain model output
with torch.no_grad():
    model_inputs = tokenizer(prompt_template.format(prompt=text), return_tensors="pt").to("cuda")

    model.generation_config.pad_token_id = model.generation_config.eos_token_id  # To prevent early ending of sentence
    outputs = model.generate(**model_inputs, max_new_tokens=256)
    # outputs = model.generate(**model_inputs, pad_token_id=tokenizer.eos_token_id)
output_text = tokenizer.batch_decode(outputs, skip_special_token=True)[0]
print(output_text)

<s> A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions.
Human: 한국의 수도는 어디인가요? 아래 선택지 중 골라주세요.

(A) 경성
(B) 부산
(C) 평양
(D) 서울
(E) 전주
Assistant:
(D) 서울이 한국의 수도입니다. 서울은 나라의 북동부에 위치해 있으며, 정치, 경제, 문화의 중심지입니다. 약 1,000만 명이 넘는 인구를 가진 세계에서 가장 큰 도시 중 하나입니다. 서울은 높은 빌딩, 현대적인 인프라, 활기찬 문화 장면으로 유명합니다. 또한, 많은 역사적 명소와 박물관이 있어 방문객들에게 풍부한 문화 체험을 제공합니다.<|im_end|>


In [5]:
# Generate tokens one by one and print them
with torch.no_grad():
    model_inputs = tokenizer(prompt_template.format(prompt=text), return_tensors="pt").to("cuda")
    output_ids = model_inputs['input_ids']
    while True:
        outputs = model.generate(output_ids, max_new_tokens=1, early_stopping=True)
        new_token_id = outputs[0, -1:]
        output_ids = torch.cat([output_ids, new_token_id.unsqueeze(0)], dim=-1)
        
        # Decode and print the latest token
        new_token = tokenizer.decode(new_token_id, skip_special_tokens=True)
        print(new_token, end=' ', flush=True)
        
        # Stop if the generated token is eos_token
        if new_token_id.item() == tokenizer.eos_token_id:
            break

/home/kimjaehyun/anaconda3/envs/nlp/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:615: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


( D ) 서울 이 한국의 수도 입니다 . 서울 은 나라의 북 동 부에 위치 해 있으며 , 정치 , 경제 , 문화 의 중심 지 입니다 . 약  1 , 0 0 0 만 명이  넘 는 인구 를 가진 세계에서 가장 큰 도시 중 하나입니다 . 서울 은  높 은 빌 딩 , 현대 적인 인 프 라 , 활 기 찬 문화 장면 으로 유명 합니다 . 또한 ,  많 은 역사적 명 소 와 박 물 관이 있어 방문 객 들에게 풍부한 문화 체 험 을 제공합니다 .  